In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

 Paths to the dataset

In [ ]:
train_path = 'C:\\Users\\sherl\\Downloads\\archive (6).zip\\dogs-vs-cat-small\\train'
test_path = 'C:\\Users\\sherl\\Downloads\\archive (6).zip\\dogs-vs-cat-small\\test'
validation_path = 'C:\\Users\\sherl\\Downloads\\archive (6).zip\\dogs-vs-cat-small\\validation'

Image parameters


In [ ]:
img_size = 64

Function to load and preprocess images

In [ ]:
def load_images_from_directory(directory, img_size):
    images = []
    labels = []
    for label, category in enumerate(['Cat', 'Dog']):
        category_path = os.path.join(directory, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


Load images from training, testing, and validation sets

In [ ]:
X_train, y_train = load_images_from_directory(train_path, img_size)
X_test, y_test = load_images_from_directory(test_path, img_size)
X_val, y_val = load_images_from_directory(validation_path, img_size)

Function to extract HOG featuresdef extract_hog_features(images):
    features = []
    for image in images:
        feature = hog(image.reshape(img_size, img_size), pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=False)
        features.append(feature)
    return np.array(features)


Extract HOG features

In [ ]:
X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)
X_val_hog = extract_hog_features(X_val)

Standardize the features and train the SVM

In [ ]:
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0, random_state=42))
svm_model.fit(X_train_hog, y_train)

Make predictions on the test set

In [ ]:
y_test_pred = svm_model.predict(X_test_hog)

Evaluate the model on the test set

In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred, target_names=['Cat', 'Dog'])

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Test Classification Report:')
print(test_report)

In [ ]:
val_accuracy = accuracy_score(y_val, y_val_pred)
val_report = classification_report(y_val, y_val_pred, target_names=['Cat', 'Dog'])
print(f'Validation Accuracy: {val_accuracy:.4f}')
print('Validation Classification Report:')
print(val_report)